- author: Lee Meng
- date: 2018-08-15 16:00
- title: 生活中的資料工程 - 如何用 Python 及 Airflow 實作一個漫畫連載通知 App
- slug: how-i-use-python-and-airflow-to-implement-a-comic-notification-app
- tags: Python, Airflow, 資料工程, Selenium, Slack
- description: Airflow 是一個以 Python 開發的工作流管理系統，也是資料工程不可或缺的利器之一。近年不管是資料科學家、資料工程師還是任何需要處理數據的軟體工程師，Airflow 都是他們用來建構 ETL 以及處理批量資料的首選之一。這篇文章希望透過 Python 及 Airflow 實作一個簡單有趣的漫畫連載通知 App 的方式，讓讀者了解 Airflow 背後的運作原理、建立資料工程的知識基礎，並在閱讀本文後發揮自己的創意，實際應用 Airflow 來解決自己以及企業的數據問題。
- summary: Airflow 是一個以 Python 開發的工作流管理系統，也是資料工程不可或缺的利器之一。近年不管是資料科學家、資料工程師還是任何需要處理數據的軟體工程師，Airflow 都是他們用來建構 ETL 以及處理批量資料的首選之一。這篇文章希望透過 Python 及 Airflow 實作一個簡單有趣的漫畫連載通知 App 的方式，讓讀者了解 Airflow 背後的運作原理、建立資料工程的知識基礎，並在閱讀本文後發揮自己的創意，實際應用 Airflow 來解決自己以及企業的數據問題。
- image: raj-eiamworakul-603747-unsplash.jpg
- image_credit_url: https://unsplash.com/photos/o4c2zoVhjSw?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText
- status: draft

[Airflow](https://airflow.apache.org/) 是一個從 Airbnb 誕生並開源，以 [Python](https://www.python.org/) 寫成的[工作流管理系統（Workflow Management System）](https://zh.wikipedia.org/wiki/%E5%B7%A5%E4%BD%9C%E6%B5%81%E7%AE%A1%E7%90%86%E7%B3%BB%E7%BB%9F)，也是[各大企業](https://github.com/apache/incubator-airflow#who-uses-airflow)的資料工程環節中不可或缺的利器之一。

近年不管是資料科學家、資料工程師還是任何需要處理數據的軟體工程師，Airflow 都是他們用來建構穩定可靠的 ETL 以及定期處理批量資料的首選之一。（事實上在 [SmartNews](https://www.smartnews.com/en/)，除了 DS/DE，會使用 Airflow 的軟體工程師也不在少數）

儘管它的方便以及強大，在完全熟悉 Airflow 之前，因為有些專業術語以及資料工程概念的存在，不少初學者（包含當時的我）在剛開始的時候容易撞壁。有些 DS 則是因為平常沒有使用需求，導致不熟悉 Airflow，等到進了企業裡頭需要建立 ETL 工作時才開始手忙腳亂。

<center>
    <a href="https://airflow.apache.org/" target="_blank">
        <img src="{filename}images/airflow/app.png" style="width:90%"/>
    </a>
    <p>[漫畫連載 App](https://github.com/leemengtaiwan/awesome-datascience-tw/blob/master/airflow/dags/notify_new_comic.py)：我們將利用 Airflow、[Selenium](http://selenium-python.readthedocs.io/)、[Slack](https://slack.com/) 來實作它（All written in [Python](https://www.python.org/)）</p>
    <br/>
</center>

這篇文章希望透過使用 Python 及 Airflow 實作一個簡單的漫畫連載通知 App，讓讀者能直觀地了解 Airflow 背後的運作原理以及學習建立資料管道（Data Pipeline）的基礎。如果你對資料工程有興趣，不太熟悉 Airflow 但有基本的 Python 程式基礎（或是純粹對我們將要實作的 App 有興趣），我相信這篇文章應該會很適合你。

P.S. 想複習 ETL 概念的讀者可以參考先前的文章：[資料科學家為何需要了解資料工程](https://leemengtaiwan.github.io/why-you-need-to-learn-data-engineering-as-a-data-scientist.html#%E8%B3%87%E6%96%99%E7%AE%A1%E9%81%93)

前言夠長了，讓我們開始吧！

## 章節傳送門
- [宅宅需求：漫畫連載通知 App](#需求：連載通知-App)
- [Python 及 Airflow 實作](#Python-及-Airflow-實作)
- [結語](#結語)

## 需求：連載通知 App

我想平常有在網路上追漫畫連載的讀者們應該都了解，市面上的漫畫網站通常都不是會員制的，更不用說「在新連載出的時候自動通知您！」這種推送功能（Push Notification）了。也因為這樣，導致我常常需要三不五時上去這些漫畫網站，看自己所有關注的漫畫到底出了最新一話了沒。可想而知，答案通常是否定的。（一週出一次每天檢查也沒用啊啊啊）

如果你只看海賊王一個漫畫（索隆好帥！），這或許沒什麼負擔。但我不只關注海賊王，還看很多其他漫畫；讓事情更糟的是，到最後你會：
- 不記得自己到底在追哪些漫畫
- 每一部漫畫最後是看到第幾話
- 上一話是什麼時候出的
- 有幾話是新出而你還沒看的

<center>
    <a href="https://airflow.apache.org/" target="_blank">
        <img src="{filename}images/airflow/tim-gouw-68319-unsplash.jpg" style="width:90%"/>
    </a>
    <p>手動追最新連載經常讓我追到懷疑人生</p>
    <br/>
</center>

這時候如果有一個 App 幫我們解決上述問題，當新連載出的時候



## Python 及 Airflow 實作 App

In [ ]:
Requirement:
Python knowledge
我鼓勵你 follow 這篇文章的步驟



Why bother airflow?
你想要做的是資料處理，而不是建立 ＷＭＳ
* Why not python script? 墨菲定律， Retry automatically
* Schedule: Corn possible, but dependencies 難以管理
* why airflow not Azakaban? Apache incubator ，簡單 UI

<center>
    <a href="https://airflow.apache.org/" target="_blank">
        <img src="{filename}images/airflow/airflow.gif" style="width:90%"/>
    </a>
    <p>除了背後實作以外，Airflow 的網頁 UI 讓資料工程師（DS） / 資料工程師（DE） 可以更輕鬆地管理及排程 ETL 工作（後面會詳細介紹）</p> 
    <br/>
</center>

## 結語

In [ ]:
在我們的漫畫 case 中 airflow 是一個 overkill, 但這篇文章希望透過go though 這個問題讓讀者熟悉 airflow 的運作，為解決企業等級的問題做好準備。

你甚至也可以選擇時用其它語言 (e.g., JAVA)


期望在讀者們能在閱讀文章之後實際應用 Airflow 來解決自己以及企業的問題。

「你也可以推薦漫畫給我」
不過老實說，我偶爾還是習慣動作會去檢查新的漫畫出來了沒。
- 結語： 你想自動化什麼？ implement it and tell me! 


之後將在奇幻旅程 2 提到我在 SmartNews 裡頭是怎麼使用 Airflow 的，歡迎訂閱